# Turbulent Transfer

Setup the gold files for turblent transfer through a random number generation. Run both the IPW `trbxfr` which implments the `hle1` calculation and the python `libsnobal.hle1`.

- H - within the resolution of the IPW outputs, difference -3.1e-4 +/- 8.3e-4
- L_v_E - within the resolution of the IPW outputs, difference -3.4e-4 +/- 1.2e-3
- E - within the resolution of the IPW outputs, difference -1.2e-10 +/- 4.8e-10

In [ ]:
import pandas as pd
import numpy as np
import subprocess as sp
import os

from pysnobal import libsnobal
from pysnobal.constants import FREEZE

def calculate_hle1(inputs):

    P_a = libsnobal.hysat(
        libsnobal.SEA_LEVEL,
        libsnobal.STD_AIRTMP,
        libsnobal.STD_LAPSE,
        inputs['elevation'] / 1000.0,
        libsnobal.GRAVITY,
        libsnobal.MOL_AIR)

    H, L_v_E, E, status = libsnobal.hle1(
        P_a,
        inputs['ta'],
        inputs['ts'],
        inputs['za'],
        inputs['ea'],
        inputs['es'],
        inputs['za'],
        inputs['wind_speed'],
        inputs['zu'],
        inputs['z0']
    )

    return H, L_v_E, E, status

In [ ]:
# Generate random inputs
nvalues = 200

df = pd.DataFrame(columns=['elevation', 'ta', 'ts', 'za',
                           'ea', 'es', 'wind_speed', 'zu', 'z0'])

df['elevation'] = np.random.randint(100, 4000, nvalues)
df['ta'] = np.random.randint(-40, 40, nvalues) + FREEZE
df['ts'] = np.random.randint(-40, 0, nvalues) + FREEZE
df['za'] = np.random.randint(2, 5, nvalues)

df['ea'] = np.random.randint(40, 7000, nvalues)
sat_vp = libsnobal.sati_np(df['ta'])
df.loc[df.ea > sat_vp, 'ea'] = sat_vp[df.ea > sat_vp]
df['es'] = libsnobal.sati_np(df['ts'])

df['wind_speed'] = np.random.randint(1, 40, nvalues)
df['zu'] = np.random.randint(3, 5, nvalues)
df['z0'] = np.random.randint(5, 100, nvalues)/1000


In [ ]:
# Run the pysnobal hle1

for index, row in df.iterrows():
    H, L_v_E, E, status = calculate_hle1(row.to_dict())

    df.loc[index, 'py_H'] = H
    df.loc[index, 'py_L_v_E'] = L_v_E
    df.loc[index, 'py_E'] = E
    df.loc[index, 'py_status'] = status

# if any failed, then remove for this purpose
df = df.loc[df.py_status != -1, :]
df.shape
    

In [ ]:
# Run trbxfr from IPW to ensure it's close

cmd_str = 'echo "{} {} {} {} {} {} {} {}" | /ipw/bin/trbxfr -z {} -d -m -K'

for index, row in df.iterrows():

    cmd = cmd_str.format(
        row['zu'],
        row['za'],
        row['z0'],
        row['ta'],
        row['ts'],
        row['ea'],
        row['es'],
        row['wind_speed'],
        row['elevation']
    )
    out = sp.check_output(cmd, shell=True, universal_newlines=True)
    ipw_out = out.rstrip().split('\t')
    
    df.loc[index, 'ipw_H'] = float(ipw_out[0])
    df.loc[index, 'ipw_L_v_E'] = float(ipw_out[1])
    df.loc[index, 'ipw_E'] = float(ipw_out[2])

df

In [ ]:
# Look at the differences between the python and IPW version

h_diff = df.py_H - df.ipw_H
lve_diff = df.py_L_v_E - df.ipw_L_v_E
e_diff = df.py_E - df.ipw_E
print(h_diff.describe())
print(lve_diff.describe())
print(e_diff.describe())

In [ ]:
df.to_csv('../pysnobal/tests/test_data_point/libsnobal/gold_turbulent_transfer.csv', index=False)